#Generates SQL which generates positive training samples and negative training material.

#Then save positive and negative samples (generated) to pickled numpy arrays.

In [159]:
%matplotlib inline
import pandas
import os
import numpy
from ordereddict import OrderedDict
from ggplot import *
import re

In [160]:
#constants for parametrizing the positive/negative sample generation
MIN_MENTIONS=2
DB_V = '233'
SAMPLE_STYLE = 'uniform'
DISTINCT='_distinct_' #change to '' if working with older reltype-other entitystats dumps.

dir_db2= '/home/db2inst1/vinith/'
dir_local= '/ceph/vinith/kg/embedding/'
standard_suffix = '%(distinct)s_v%(db_v)s_minmentions%(min_mentions)i_style%(style)s' % {'distinct':DISTINCT,'db_v':DB_V, 'min_mentions':MIN_MENTIONS, 'style':SAMPLE_STYLE}


##Dump relations from DB2

In [161]:
dump_path = dir_db2+'dump_v%(db_v)s_minmentions%(min_mentions)s.del' %{"db_v":DB_V,"min_mentions":MIN_MENTIONS}
sql = '''
connect to KG_R%(db_v)s;
export to %(dump_path)s of del
SELECT * FROM
(Select 
    "subject/instanceName_up",
    "subject/typeType",
    "object/instanceName_up",
    "object/typeType",
    "relCanonicalName",
    SUM("mAB_doc") as "mAB",
    count(*) as "n_docs"
from (
     Select
        "subject/instanceName_up",
        "subject/typeType",
        "object/instanceName_up",
        "object/typeType",
        "relCanonicalName",
        "documentID",
        count(*) as "mAB_doc"
     from 
     (select * from ADS."SIRE1.1"
     fetch first %(row_limit)i rows only)
     group by ( "subject/instanceName_up",
                "subject/typeType",
                "object/instanceName_up",
                "object/typeType",
                "relCanonicalName",
                "documentID"
              )
     )
group by (  "subject/instanceName_up",
            "subject/typeType",
            "object/instanceName_up",
            "object/typeType",
            "relCanonicalName"
         )
)
Where "mAB" > %(min_mab)i;''' % {"db_v":DB_V,"dump_path": dump_path, "min_mab": MIN_MENTIONS-1, "row_limit": 300000000}

print sql


connect to KG_R233;
export to /home/db2inst1/vinith/dump_v233_minmentions2.del of del
SELECT * FROM
(Select 
    "subject/instanceName_up",
    "subject/typeType",
    "object/instanceName_up",
    "object/typeType",
    "relCanonicalName",
    SUM("mAB_doc") as "mAB",
    count(*) as "n_docs"
from (
     Select
        "subject/instanceName_up",
        "subject/typeType",
        "object/instanceName_up",
        "object/typeType",
        "relCanonicalName",
        "documentID",
        count(*) as "mAB_doc"
     from 
     (select * from ADS."SIRE1.1"
     fetch first 300000000 rows only)
     group by ( "subject/instanceName_up",
                "subject/typeType",
                "object/instanceName_up",
                "object/typeType",
                "relCanonicalName",
                "documentID"
              )
     )
group by (  "subject/instanceName_up",
            "subject/typeType",
            "object/instanceName_up",
            "object/typeType",
            "r

###Run the above SQL in db2 on wd-app03.
###Then, copy over the file to the path: local_path_reltable

##Dump Entity data from DB2

In [162]:
sql = '''
export to /home/db2inst1/vinith/dump_entities_distinct_v%(db_v)s_minmentions%(min_mentions)i.del of del
Select DISTINCT "ENTITYNAME","ENTITYTYPE" from ADS.ENTITYSTATS2
where "RELATIONSHIPMENTIONCOUNT" > %(min_mentions_minusone)i
fetch first %(row_limit)i rows only;
''' % {"db_v":DB_V, "min_mentions":MIN_MENTIONS,"min_mentions_minusone":MIN_MENTIONS-1, "row_limit":100000000}
print sql


export to /home/db2inst1/vinith/dump_entities_distinct_v233_minmentions2.del of del
Select DISTINCT "ENTITYNAME","ENTITYTYPE" from ADS.ENTITYSTATS2
where "RELATIONSHIPMENTIONCOUNT" > 1
fetch first 100000000 rows only;



###Run the above SQL on wd-app03
###Copy over file to local_path_entitytable

##load both tables into pandas

In [163]:
relcols = ['A',
           'At',
           'B',
           'Bt',
           'reltype',
           'mAB',
           'nDocs']
local_path_reltable = '/ceph/vinith/kg/embedding/dump_v%(db_v)s_minmentions%(min_mentions)s.del' %{"db_v":DB_V,"min_mentions":MIN_MENTIONS}
reldata = pandas.read_csv(local_path_reltable,header=None,names=relcols,nrows=None,na_filter=True).dropna()

In [164]:
entitycols = ['A',
              'At']
local_path_entitytable = '/ceph/vinith/kg/embedding/dump_entities%(distinct)sv%(db_v)s_minmentions%(min_mentions)i.del' %{"distinct":DISTINCT, "db_v":DB_V,'min_mentions':MIN_MENTIONS}
entitydata = pandas.read_csv(local_path_entitytable,header=None,names=entitycols,na_filter=False)
len(entitydata)

24209271

In [165]:
print entitydata.ix[:10,:]
#print reldata.ix[:,:]

                                                    A      At
0                                                      PERSON
1                                                 ...  PERSON
2                                               CONAN  PERSON
3                                       SABELLA SABLE  PERSON
4                                             A'KEIBA  PERSON
5    AARHUS - FLENSBURG - HAMBURG - BERLIN - DRESD...  PERSON
6                                             AARSETH  PERSON
7                                              ABWEHR  PERSON
8                                               ADELE  PERSON
9                                           AEROSMITH  PERSON
10                                          AETHIOPIS  PERSON


In [166]:
entitydata['idx'] = pandas.Series(range(len(entitydata)))
entitydata.set_index(['A','At'],inplace=True)
entity_to_index = entitydata.to_dict()['idx']

In [167]:
import pdb
def Atest(row):
    key = (row['A'].strip(),row['At'])
    try:
        return entity_to_index[key]
    except KeyError,e:
        #print 'error'
        return -1
def Btest(row):
    key = (row['B'].strip(),row['Bt'])
    try:
        return entity_to_index[key]
    except KeyError,e:
        #print 'error'
        return -1

In [168]:
reldata

,A,At,B,Bt,reltype,mAB,nDocs
167,CONAN,PERSON,WRATH-AMON,TITLEWORK,colocation,2,1
168,ADELE,PERSON,SOMEONE LIKE YOU,TITLEWORK,authorOf,3,3
169,AIDA,PERSON,AMNERIS,PERSON,colocation,2,1
170,AIDA,PERSON,LA MONNAIE,GPE,colocation,2,1
171,ALBENA,PERSON,SINGLES,EVENT_SPORTS,participantIn,2,2
172,ALICIA KEYS,PERSON,THE DIARY OF ALICIA KEYS,TITLEWORK,authorOf,2,1
173,ALLEN,PERSON,CURT SCHNEIDER HARRY FOX AGENCY,PERSON,colocation,3,1
174,ALLEN,PERSON,LINDY ROBBINS HEINZ WINCKLER,PERSON,colocation,2,1
175,ALLEN,PERSON,MARY STEENBURGEN HARRY FOX AGENCY,PERSON,colocation,4,1
176,ALMATY,PERSON,KAZAKHSTAN,GPE,locatedAt,3,3


In [169]:
reldata['Aindex'] = reldata.apply(func = Atest,axis=1)
reldata['Bindex'] = reldata.apply(func = Btest,axis=1)

In [170]:
reldata['label'] = 1
reldata.drop(labels=['At','Bt','reltype','mAB','nDocs'],axis = 1, inplace=True)
reldata = reldata[(~(reldata['Aindex']==-1))&(~(reldata['Bindex']==-1))]
print len(reldata)

22886956


In [171]:
len(reldata)

22886956

##Positive Sampling: reldata's A and B fields, as is

##Negative Sampling: for now pick randomly, but leave door open to do something more involved.

In [172]:
from numpy.random import randint,permutation
reldata_neg = reldata.copy()

#first, flip label
reldata_neg['label'] = -1

#next, generate noisy entity to be swapped in for each sample
new_entity = randint(0,len(entitydata),len(reldata_neg))

#swap half of the rows at random
rows = range(len(reldata_neg))
rows = permutation(rows)

Aarray = numpy.asarray(reldata_neg.ix[:,'Aindex'])
Barray = numpy.asarray(reldata_neg.ix[:,'Bindex'])

Aarray[rows[:len(rows)/2]] = new_entity[:len(rows)/2]
Barray[rows[len(rows)/2:]] = new_entity[len(rows)/2:]

reldata_neg['Aindex'] = Aarray
reldata_neg['Bindex'] = Barray

In [173]:
print rows[:10]
print reldata[:10]
print reldata_neg[:10]

[22329661 17387001 21751956 21559961 14307023  6463376 10275000 15896382
 19712791  4715766]
                               A                                  B   Aindex  \
167                        CONAN                         WRATH-AMON  4771229   
168                        ADELE                   SOMEONE LIKE YOU   593975   
169                         AIDA                            AMNERIS   736123   
170                         AIDA                         LA MONNAIE   736123   
171                       ALBENA                            SINGLES   869244   
172                  ALICIA KEYS           THE DIARY OF ALICIA KEYS  1003418   
173                        ALLEN    CURT SCHNEIDER HARRY FOX AGENCY  1038214   
174                        ALLEN       LINDY ROBBINS HEINZ WINCKLER  1038214   
175                        ALLEN  MARY STEENBURGEN HARRY FOX AGENCY  1038214   
176                       ALMATY                         KAZAKHSTAN  1060974   

       Bindex  label  
167

##Dump data to CSV, and then copy over to db2 machine for featurization

In [174]:
#data_x = numpy.concatenate([reldata[['Aindex','Bindex']].as_matrix(),reldata_neg[['Aindex','Bindex']].as_matrix()],axis=0)
#data_y = numpy.concatenate([reldata[['label']].as_matrix(),reldata_neg[['label']].as_matrix()],axis=0)
data_x1 = reldata[['Aindex','Bindex']].as_matrix()
data_x0 = reldata_neg[['Aindex','Bindex']].as_matrix()
data_y1 = reldata[['label']].as_matrix()
data_y0 = reldata_neg[['label']].as_matrix()

In [204]:
test = pandas.unique(reldata[['Aindex','Bindex']].values.flatten())

In [205]:
len(test)

5336391

In [175]:
import cPickle as pickle
pickle.dump([data_x1,data_x0,data_y1,data_y0],open(dir_local+'traindata'+standard_suffix+'.pkl','w'))

In [176]:
print dir_local+'traindata'+standard_suffix+'.pkl'

/ceph/vinith/kg/embedding/traindata_distinct__v233_minmentions2_styleuniform.pkl
